In [1]:
from datetime import timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd

import sys
sys.path.insert(1, '../src')
from feature_extractor import preprocess_features, aggregate_customer_features, iv_woe

In [2]:
def calculate_rfm(df, snaphot_date=None):
    """
    Calculate RFM (Recency, Frequency, Monetary) features from a DataFrame.
    
    Parameters:
    df (DataFrame): Input DataFrame containing 'CustomerId', 'transaction_date', and 'amount'.
    snaphot_date (datetime, optional): The date to consider as the snapshot for recency calculation.
    
    Returns:
    DataFrame: A DataFrame with RFM features.
    """
    if snaphot_date is None:
        snaphot_date = df['TransactionStartTime'].max() + timedelta(days=1)
    
    rfm = df.groupby('CustomerId').agg({
        'TransactionStartTime': lambda x: (snaphot_date - x.max()).days,
        'TransactionId': 'count',
        'Amount': 'sum'
    }).rename(
        columns={
            'TransactionStartTime': 'Recency',
            'TransactionId': 'Frequency',
            'Amount': 'Monetary'
        }
    ).reset_index()
    
    rfm.columns = ['CustomerId', 'Recency', 'Frequency', 'Monetary']
    
    return rfm

In [3]:
def scale_rfm(rfm_df):
    rfm_values = rfm_df[['Recency', 'Frequency', 'Monetary']]
    scaler = StandardScaler()
    scaled_rfm = scaler.fit_transform(rfm_values)

    return scaled_rfm, scaler

In [4]:
def create_rfm_clusters(scaled_rfm, random_state=42):
    kmeans = KMeans(n_clusters=3, random_state=random_state)
    clusters = kmeans.fit_predict(scaled_rfm)
    return clusters

In [5]:
def assign_proxy_label(rfm_df, clusters):
    rfm_df['cluster'] = clusters

    # Compute mean values per cluster to identify least engaged one
    cluster_profile = rfm_df.groupby('cluster')[['Recency', 'Frequency', 'Monetary']].mean().reset_index()
    print (cluster_profile)

    # Let's assume lowest frequency & monitary = high risk

    high_risk_cluster = cluster_profile.sort_values(by=['Frequency', 'Monetary']).index[0]
    rfm_df['is_high_risk'] = (rfm_df['cluster'] == high_risk_cluster).astype(int)

    return rfm_df

In [6]:
df = pd.read_csv('../data/raw/data.csv')
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

In [7]:
rfm = calculate_rfm(df)
scale_rfm, scaler = scale_rfm(rfm)


In [8]:
clusters = create_rfm_clusters(scale_rfm)
rfm_labeled = assign_proxy_label(rfm, clusters)
df_labeled = df.merge(rfm_labeled[['CustomerId', 'is_high_risk']], on='CustomerId', how='left')

   cluster    Recency    Frequency      Monetary
0        0  61.859846     7.726699  8.172379e+04
1        1  29.000000  4091.000000 -1.049000e+08
2        2  12.716076    34.807692  2.726546e+05


In [9]:
df_labeled.sample(5)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,is_high_risk
45670,TransactionId_134070,BatchId_120673,AccountId_4841,SubscriptionId_3829,CustomerId_3259,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-5000.0,5000,2018-12-28 21:09:53+00:00,2,0,1
25855,TransactionId_108126,BatchId_80047,AccountId_2314,SubscriptionId_4722,CustomerId_2728,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1500.0,1500,2018-12-14 06:00:58+00:00,2,0,0
61556,TransactionId_70966,BatchId_11914,AccountId_2040,SubscriptionId_2323,CustomerId_2441,UGX,256,ProviderId_1,ProductId_10,airtime,ChannelId_3,1000.0,1000,2019-01-16 08:04:28+00:00,4,0,0
80669,TransactionId_61985,BatchId_79277,AccountId_4841,SubscriptionId_3829,CustomerId_1260,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-5000.0,5000,2019-02-01 09:13:41+00:00,2,0,0
35397,TransactionId_37785,BatchId_78833,AccountId_4841,SubscriptionId_3829,CustomerId_4834,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-5000.0,5000,2018-12-21 12:15:33+00:00,2,0,0


In [10]:
processed_df, _ = preprocess_features(df_labeled)

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult',
       'is_high_risk'],
      dtype='object')


In [28]:
processed_df.to_csv('../data/processed/processed_data.csv', index=False)

In [30]:
processed_df['is_high_risk_1'].value_counts()

is_high_risk_1
0.0    94614
1.0     1048
Name: count, dtype: int64

In [31]:
new_df, woe_df = iv_woe(processed_df, 'is_high_risk_1')

Information value of Amount is 0.032123
Information value of Value is 0.158736
Information value of ProductCategory_airtime is 0.021511
Information value of ProductCategory_data_bundles is 0.018351
Information value of ProductCategory_financial_services is 5.80358
Information value of ProductCategory_movies is 1.399147
Information value of ProductCategory_other is 0.518134
Information value of ProductCategory_ticket is 0.096997
Information value of ProductCategory_transport is 3.628243
Information value of ProductCategory_tv is 3e-06
Information value of ProductCategory_utility_bill is 0.001421
Information value of ProviderId_ProviderId_1 is 0.002832
Information value of ProviderId_ProviderId_2 is 0.000126
Information value of ProviderId_ProviderId_3 is 0.022446
Information value of ProviderId_ProviderId_4 is 0.096877
Information value of ProviderId_ProviderId_5 is 0.289341
Information value of ProviderId_ProviderId_6 is 0.000264
Information value of ProductId_ProductId_1 is 0.136755
I

In [47]:
woe_df.head(15).sort_values(by='IV', ascending=False)

,Variable,Cutoff,N,Events,% of Events,Non-Events,% of Non-Events,WoE,IV
4,Value,4.0,27716,146.0,0.139313,27570.0,0.291395,0.737955,1.122293e-01
2,Value,2.0,10232,159.0,0.151718,10073.0,0.106464,-0.354212,1.602931e-02
3,Amount,"(14.0, 16.0]",13591,104.0,0.099237,13487.0,0.142548,0.362169,1.568588e-02
1,Value,1.0,15049,210.0,0.200382,14839.0,0.156837,-0.245015,1.066905e-02
3,Value,3.0,11039,158.0,0.150763,10881.0,0.115004,-0.270743,9.681580e-03
0,Value,0.0,11837,165.0,0.157443,11672.0,0.123364,-0.243919,8.312364e-03
0,Amount,"(-0.001, 6.0]",13162,179.0,0.170802,12983.0,0.137221,-0.218912,7.351231e-03
6,Amount,"(19.0, 23.0]",5472,74.0,0.070611,5398.0,0.057053,-0.213203,2.890570e-03
7,Amount,"(6.0, 8.0]",11362,144.0,0.137405,11218.0,0.118566,-0.147460,2.777947e-03
1,Amount,"(11.0, 13.0]",10486,102.0,0.097328,10384.0,0.109751,0.120127,1.492329e-03
